![email inbox](email_inbox.jpg)

Every day, professionals wade through hundreds of emails, from urgent client requests to promotional offers. It's like trying to find important messages in a digital ocean. But AI can help you stay afloat by automatically sorting emails to highlight what matters most.

You've been asked to build an intelligent email assistant using Llama, to help users automatically classify their incoming emails. Your system will identify which emails need immediate attention, which are regular updates, and which are promotions that can wait or be archived.

### The Data
You'll work with a dataset of various email examples, ranging from urgent business communications to promotional offers. Here's a peek at what you'll be working with:

### email_categories_data.csv

 Column | Description |
|--------|-------------|
| email_id | A unique identifier for each email in the dataset. |
| email_content | The full email text including subject line and body. Each email follows a format of "Subject" followed by the message content on a new line. |
| expected_category | The correct classification of the email: `Priority`, `Updates`, or `Promotions`. This will be used to validate your model's performance. |



In [80]:
# Run the following cells first
# Install necessary packages
!pip install llama-cpp-python==0.2.82 -q -q -q

In [81]:
# Download the model
!wget -q https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v0.3-GGUF/resolve/main/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf?download=true -O model.gguf

In [82]:
# Import required libraries
import pandas as pd
from llama_cpp import Llama

In [83]:
# Load the email dataset
emails_df = pd.read_csv('data/email_categories_data.csv')
# Display the first few rows of our dataset
print("Preview of our email dataset:")
emails_df.head(2)

Preview of our email dataset:


,email_id,email_content,expected_category
0,1,Urgent: Server Maintenance Required\nOur main ...,Priority
1,2,50% Off Spring Collection!\nDon't miss our big...,Promotions


In [84]:
# Initialize the Llama model
model_path = "model.gguf"
llm = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 20 key-value pairs and 201 tensors from model.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = py007_tinyllama-1.1b-chat-v0.3
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.head_count u32             

In [85]:
# The prompt
prompt = """

You classify emails into Priority, Updates, or Promotions.

Example 1:
Urgent: Password Reset Required
Your account security requires immediate attention. Please reset your password within 24 hours.
Response:Priority

Example 2:
Special Offer - 50% Off Everything!
Don't miss our biggest sale of the year. Everything must go!
Response: Promotions

Example 3:
Canceled Event - Team Meeting
This event has been canceled and removed from your calendar.
Response: Updates

Example 4:
"""

In [86]:
# Process messages and return classification
def process_message(llm, message, prompt):
    """Process message and return response"""
    
    input_prompt = f"{prompt} {message}\nResponse:"
    response = llm(
        input_prompt,
        max_tokens=5,
        temperature=0,
        stop=["Q:", "\n"]
    )
    
    return response['choices'][0]['text'].strip()

In [87]:
# Testing the classifier for two emails
test_emails = emails_df.head(2)
test_emails

,email_id,email_content,expected_category
0,1,Urgent: Server Maintenance Required\nOur main ...,Priority
1,2,50% Off Spring Collection!\nDon't miss our big...,Promotions


In [88]:
# Process each test email and store results
results = []
for idx, row in test_emails.iterrows():
    email_content = row['email_content']
    expected_category = row['expected_category']
    
    # Get model's classification
    result = process_message(llm, email_content, prompt)
    
    # Store results
    results.append({
        'email_content': email_content,
        'expected_category': expected_category,
        'model_result': result
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)


llama_print_timings:        load time =    8129.52 ms
llama_print_timings:      sample time =       0.67 ms /     3 runs   (    0.22 ms per token,  4491.02 tokens per second)
llama_print_timings: prompt eval time =    8129.31 ms /   171 tokens (   47.54 ms per token,    21.03 tokens per second)
llama_print_timings:        eval time =     286.88 ms /     2 runs   (  143.44 ms per token,     6.97 tokens per second)
llama_print_timings:       total time =    8418.85 ms /   173 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    8129.52 ms
llama_print_timings:      sample time =       0.63 ms /     4 runs   (    0.16 ms per token,  6329.11 tokens per second)
llama_print_timings: prompt eval time =    2801.81 ms /    33 tokens (   84.90 ms per token,    11.78 tokens per second)
llama_print_timings:        eval time =     309.11 ms /     3 runs   (  103.04 ms per token,     9.71 tokens per second)
llama_print_timings:       total time =    3112.87 ms /    36 

In [91]:
result1 = results_df['model_result'][0]
result2 = results_df['model_result'][1]

# Display results
print(f"\nClassification Results: \n email 1 {result1} \n email_2 {result2}")


Classification Results: 
 email 1 Priority 
 email_2 Promotions
